In [1]:
import pandas as pd
import ppscore as pp
import re

In [2]:
df_forecast = pd.read_csv("forecast_starting_data.csv")
df_training = pd.read_csv("training_data.csv")
df_macro = pd.read_csv("macro_data.csv")

/var/folders/jh/hbpckc3j0vj731hydfqf993h0000gn/T/ipykernel_21558/2315000146.py:2: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_training = pd.read_csv("training_data.csv")


IN THE NEXT STEP WE WILL FILTER THE MACRO DATA

In [188]:
def add_mths(input: str, n: int):
    input = (int)(input)
    last_two = input % 100
    input -= last_two
    if last_two + n > 12:
        input += 100
    input += (last_two + n) % 12
    return (str)(input)
    
def format_date(input: str):
    x = ""
    if len(input) == 9:
        x = input[-4:] + "0" + input[0]
    elif len(input) == 10:
        x = input[-4:] + input[0:2]
    else:
        return "YOU BEEN A VERY BAD BOY TODAY >:("
    return add_mths(x, 6)
    
# TESTING FORMAT_DATE()
print(format_date("12/31/2020"))
print(format_date("2/31/2020"))
print(format_date("2/31/20"))

202106
202008
YOU BEEN A VERY BAD BOY TODAY >:(


In [189]:
import numpy as np
import datetime

In [190]:
df_macro

,Mnemonic,M_FHOFHOPIPOQ.IUSA,M_FYLE.IUSA,M_FRT.IUSA,M_FYPSAVQ.IUSA,M_FC.IUSA,FYPCPICA_FEDB.IUSA,M_FGDP.IUSA,M_FYHHAVGQ.IUSA,M_FYFMMEDQ.IUSA,...,M_FBCFCOFCCQ.IUSA,M_FETP.IUSA,M_FLTV.IUSA,M_FC$.IUSA,M_FEIAMGTTQ.IUSA,M_FXCPMGASUQ.IUSA,M_FRBBBQ.IUSA,M_FHOWNRQ.IUSA,M_FIFR_FEDS.IUSA,M_FIFR$_FEDS.IUSA
0,Description,Baseline Scenario (October 2022): FHFA Purchas...,Baseline Scenario (October 2022): NIPA: Income...,Baseline Scenario (October 2022): Retail Sales...,Baseline Scenario (October 2022): Income: Pers...,Baseline Scenario (October 2022): NIPA: Person...,Baseline Scenario (October 2022): Income: Per ...,Baseline Scenario (October 2022): NIPA: Gross ...,Baseline Scenario (October 2022): Average Hous...,Baseline Scenario (October 2022): Median Famil...,...,Baseline Scenario (June 2020): Charge Off Rate...,Baseline Scenario (June 2020): Employment: Tot...,Baseline Scenario (June 2020): Terms Conventio...,Baseline Scenario (June 2020): NIPA: Personal ...,Baseline Scenario (June 2020): U.S. All Grades...,Baseline Scenario (June 2020): Motor gasoline ...,Baseline Scenario (June 2020): Merrill Lynch 1...,Baseline Scenario (June 2020): Homeownership R...,FRB CCAR 2020 - Severely Adverse: NIPA: Fixed ...,FRB CCAR 2020 - Severely Adverse: NIPA: Fixed ...
1,Name,"FHFA Purchase-Only Home Price Index, (Index 19...","NIPA: Income - Compensation of Employees, (Bil...","Retail Sales - Retail Sales Total, (Bil. USD, ...","Income: Personal - Saving Rate, (%, SA)",NIPA: Personal Consumption Expenditures - Tota...,"Income: Per Capita Income, (USD)","NIPA: Gross Domestic Product, (Bil. USD, SAAR)","Average Household Income: All Races, (USD, SA)","Median Family Income: All Races, (USD, SA)",...,Charge Off Rates: All Commercial Banks - Credi...,"Employment: Total Private, (Mil. #, SA)",Terms Conventional Mortgages: All Loans - Comp...,NIPA: Personal Consumption Expenditures - Tota...,U.S. All Grades All Formulations Retail Gasoli...,"Motor gasoline prices: Unleaded, (Cents per ga...","Merrill Lynch 10-Year Bbb Corporate Bond Rate,...","Homeownership Rate, (%, SA)","NIPA: Fixed Investment - Residential - Total, ...","NIPA: Fixed Investment - Residential - Total, ..."
2,Source,U.S. Federal Housing Finance Agency (FHFA); Fr...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Census Bureau (BOC); Moody's Analytics Fo...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Census Bureau (BOC); U.S. Bureau of Econo...,U.S. Census Bureau (BOC); Moody's Analytics Fo...,...,U.S. Board of Governors of the Federal Reserve...,U.S. Bureau of Labor Statistics (BLS); Moody's...,U.S. Federal Housing Finance Board (FHFB); Moo...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Energy Information Administration (EIA); ...,Oil Price Information Service (OPIS); Moody's ...,U.S. Board of Governors of the Federal Reserve...,U.S. Census Bureau (BOC); Moody's Analytics Fo...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...
3,Native Frequency,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,ANNUAL,MONTHLY,MONTHLY,MONTHLY,...,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY
4,Geography,United States,United States,United States,United States,United States,United States,United States,United States,United States,...,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,8/31/2035,517.343026,24477.50421,10683.55223,5.850066485,30258.36203,109631.1675,44455.57687,272055.4718,138308.4502,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,9/30/2035,518.9847037,24562.9831,10715.48389,5.856203152,30357.61418,109972.5749,44603.62923,272841.9069,138661.205,...,NaN,NaN,NaN,NaN,NaN,NaN

In [191]:
macro_final = pd.DataFrame()
for i, row in df_macro.iterrows():
    if re.search(r"2017$", str(row["Mnemonic"])) or re.search(r"2018$", str(row["Mnemonic"])) or re.search(r"2019$", str(row["Mnemonic"])) or re.search(r"2020$", str(row["Mnemonic"])) or re.search(r"2021$", str(row["Mnemonic"])):
        t1 = pd.DataFrame(row).T
        month_code = format_date(t1['Mnemonic'].iloc[0])
        t1['mth'] = month_code
        macro_final = pd.concat([macro_final, t1], axis=0)
            
print(set(macro_final.columns).intersection(df_clean.columns))

set()


NEXT WE WILL STRATIFY SAMPLE THE CHARGED AND NON-CHARGED DATA

In [192]:
total_num = len(df_training)
charge_num = 0
for i in range(len(df_training)):
    if df_training["charge_off"][i] != 0:
        charge_num += 1


In [193]:
df_no_charge = df_training[df_training["charge_off"] == 0] 
df_no_charge_sample = df_no_charge.sample(n=15698)
df_charge = df_training[df_training["charge_off"] == 1]
df_clean = pd.concat([df_charge, df_no_charge_sample], axis=0)

In [194]:

x = pd.DataFrame()
final = pd.DataFrame()
a = 0 #
for i in df_clean['mth_code'].unique():
    left_block = df_clean[df_clean['mth_code'] == i]
    temp2 = macro_final[macro_final['mth'] == ((str)(i))]
    right_block = pd.concat([temp2]*len(left_block))
    
    left_block = left_block.reset_index(drop=True)
    right_block = right_block.reset_index(drop=True)
    
    x = pd.concat([left_block, right_block], axis=1)
    if a == 0:
        final = x
        a+=1
    else: 
        final = pd.concat([x, final], axis=0)
final
    

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,charge_off_reason_code,mob,open_closed_flag,...,M_FETP.IUSA,M_FLTV.IUSA,M_FC$.IUSA,M_FEIAMGTTQ.IUSA,M_FXCPMGASUQ.IUSA,M_FRBBBQ.IUSA,M_FHOWNRQ.IUSA,M_FIFR_FEDS.IUSA,M_FIFR$_FEDS.IUSA,mth
0,1.0,T,0,0,I 7-12,1.0,721-740,NaN,27,1.0,...,124.368,79.5,12551.989,2.4144,229.356,3.906289724,63.90535443,747.9049448,602.8415398,201801
1,0.0,R,0,0,A,1.0,701-720,NaN,16,1.0,...,124.368,79.5,12551.989,2.4144,229.356,3.906289724,63.90535443,747.9049448,602.8415398,201801
2,0.0,P,1,1,I 0-6,1.0,641-660,NaN,65,1.0,...,124.368,79.5,12551.989,2.4144,229.356,3.906289724,63.90535443,747.9049448,602.8415398,201801
3,1.0,I,0,1,A,0.0,581-600,NaN,56,1.0,...,124.368,79.5,12551.989,2.4144,229.356,3.906289724,63.90535443,747.9049448,602.8415398,201801
4,1.0,T,0,0,A,1.0,581-600,NaN,4,1.0,...,124.368,79.5,12551.989,2.4144,229.356,3.906289724,63.90535443,747.9049448,602.8415398,201801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0.0,R,1,0,I 13-24,1.0,761+,NaN,49,1.0,...,124.554,79.5,12564.612,2.494,237.2825,3.883683951,63.89362222,751.899472,604.399634,201802
186,1.0,I,0,1,A,1.0,761+,NaN,8,1.0,...,124.554,79.5,12564.612,2.494,237.2825,3.883683951,63.89362222,751.899472,604.399634,201802
187,1.0,T,1,0,A,0.0,661-680,NaN,3,1.0,...,124.554,79.5,12564.612,2.494,237.2825,3.883683951,63.89362222,751.899472,604.399634,201802
188,0.0,P,0,0,I 13-24,1.0,761+,NaN,79,1.0,...,124.554,79.5,12564.612,2.494,237.2825,3.883683951,63.89362222,751.899472,604.399634,201802


In [195]:
data = []
for column in final.columns:
    data.append(pp.score(final, x=column, y="charge_off"))
pd.DataFrame(data).sort_values(by="ppscore", ascending=False).reset_index().head(n=50)


,index,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
0,27,total_writeoff_amt,charge_off,1.000000,regression,True,mean absolute error,0.4944,0.000000,DecisionTreeRegressor()
1,21,active,charge_off,1.000000,regression,True,mean absolute error,0.4944,0.000000,DecisionTreeRegressor()
2,22,charge_off,charge_off,1.000000,predict_itself,True,None,0.0000,1.000000,None
3,50,writeoff_type_null,charge_off,0.995954,regression,True,mean absolute error,0.4944,0.002000,DecisionTreeRegressor()
4,26,principal_amt_chrg_off,charge_off,0.976788,regression,True,mean absolute error,0.4944,0.011476,DecisionTreeRegressor()
5,28,fee_chg_off_reversal_amt,charge_off,0.877639,regression,True,mean absolute error,0.4944,0.060495,DecisionTreeRegressor()
6,35,fc_reversals,charge_off,0.839196,regression,True,mean absolute error,0.4944,0.079502,DecisionTreeRegressor()
7,23,charge_off_aged,charge_off,0.799762,regression,True,mean absolute error,0.4944,0.098998,DecisionTreeRegressor()
8,36,fee_reversals,charge_off,0.793146,regression,True,mean absolute error,0.4944,0.102269,DecisionTreeRegressor()
9,18,credit_limit_amt,charge_off,0.686379,regression,True,mean absolute error,0.4944,0.155054,DecisionTreeRegressor()


In [196]:
corr = final["charge_off"].corr(final["M_FCCDBC.IUSA"].astype(float))
corr


-0.0395263302450835